In [2]:
import pandas as pd
from scipy.io import loadmat
from dtaidistance import dtw_ndim
from dtaidistance import dtw_ndim_visualisation
import dtw
from tqdm import tqdm
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn_extra.cluster import KMedoids

CROSS数据集

In [17]:
tracks = np.load("../CROSS_dataset/cross/tracs.npy", allow_pickle=True)
for i in range(tracks.shape[0]):
    tracks[i][0] = tracks[i][0].transpose([1, 0])
labels = np.load("../CROSS_dataset/cross/labels.npy", allow_pickle=True)
labels.flatten
print(tracks.shape)
print(labels.shape)

(200, 1)
(200,)


In [19]:
dis_mat = np.zeros([tracks.shape[0], tracks.shape[0]])
with tqdm(total=tracks.shape[0]) as pbar1:
    for num1,track1 in enumerate(tracks):
        for num2,track2 in enumerate(tracks):
            if num1 < num2:
                dis=dtw_ndim.distance(track1[0], track2[0])
                dis_mat[num1,num2]=dis
                dis_mat[num2,num1]=dis
        pbar1.update(1)
np.save("CROSS_dis_mat.npy",dis_mat)

100%|██████████| 200/200 [00:36<00:00,  5.43it/s]


In [20]:
#similarity = np.exp(-beta * distance / distance.std())
sim_mat = np.exp(-0.5 * dis_mat / dis_mat.std())
np.save("CROSS_sim_mat.npy", sim_mat)

In [21]:
dis_mat=np.load("CROSS_dis_mat.npy")
sim_mat=np.load("CROSS_sim_mat.npy")

聚类算法实现

In [22]:
sse_max = -1
best_k = 0
sse_list = []
for k in range(2, 21):
    res = SpectralClustering(n_clusters=k, affinity='precomputed').fit(sim_mat)
    tmp = sklearn.metrics.silhouette_score(X=dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_k = k
best_res = SpectralClustering(n_clusters=best_k, affinity='precomputed').fit(sim_mat)
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=best_res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=best_res.labels_)

print("sse_max:", sse_max)
print("best_k:", best_k)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.82581094775327
best_k: 13
nmi: 0.9256235676432637
ari: 0.7542315718745844


In [24]:
norm_dis_mat = dis_mat / (dis_mat.max() - dis_mat.min()) * 1000.0
#print(norm_dis_mat)
best_eps = -1
sse_max = -1
sse_list = []
for r in range(30, 200):
    res = DBSCAN(eps=r, metric='precomputed').fit(norm_dis_mat)
    #print(res.labels_)
    tmp = sklearn.metrics.silhouette_score(X=norm_dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_eps = r
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=res.labels_)
print("sse_max:", sse_max)
print("best_eps:", best_eps)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.82581094775327
best_eps: 68
nmi: 0.9256235676432637
ari: 0.7542315718745844


In [25]:
sse_max = -1
best_k = 0
sse_list = []
for k in range(2, 21):
    res = KMedoids(n_clusters=k, metric='precomputed').fit(dis_mat)
    tmp = sklearn.metrics.silhouette_score(X=dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_k = k
best_res = KMedoids(n_clusters=best_k, metric='precomputed').fit(dis_mat)
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=best_res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=best_res.labels_)

print("sse_max:", sse_max)
print("best_k:", best_k)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.6125915041861852
best_k: 8
nmi: 0.7990073449906595
ari: 0.476636096694667
